#Specifiche richieste nel paper

The fit_longitudinal_auto_encoder.py file is the main script that should be run. To run it, one needs

1. A data_dict dictionnary that contains data_dict['data'] **tensor of dimension N_{samples} x  W x L x D will all the 3D images**
2. A data_dict['timepoints'] with **all the ages** of subjects at the times of visits.
3. A data_dict['label'] that contains the **ID of the patients**.
4. A model.xml file that contains **all the training parameters and paths to save the model**. This is especially useful if one want's to load a pre-trained model and pass all the parameters as arguments.
5. In the main function, one should also give the path to the data_dict file and the output folder.

If the convolutional network is not pre-trained, the training starts by training a regular VAE with the data, before adding the longitudinal structure later on, with the learned convolutional weights.

# Specifiche fornite dal creatore

- Il campo 'data' contiene i tensori effettivi delle immagini.
- I timepoint sono le età dei soggetti a ogni visita.
- Il 'label' è l'ID (ad esempio il nome del paziente o semplicemente un numero).

Per esempio
* Dati: ['immagine1delpaziente1', 'immagine2delpaziente1', 'immagine1delpaziente2', 'immagine2delpaziente2'].

* Punti temporali: [74, 75, 81, 82] (le età dei pazienti 1 e 2 in entrambe le visite)

* Etichetta: [1, 1, 2, 2] (l'ID del paziente per ogni visita)

La rete può anche essere preaddestrata senza perdita longitudinale per iniziare con una VAE decente, basta impostare lambda a 0 o scrivere un piccolo script che riqualifichi una VAE regolare.

In [1]:
# Load the Drive helper and mount
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive/')

Mounted at /content/drive/


#Formato Nib

Nonostante nel file xml ci sia scritto che:


---

\begin{verbatim}
<protocol term="Matrix X">240.0</protocol>
<protocol term="Matrix Y">256.0</protocol>
<protocol term="Matrix Z">208.0</protocol>
\end{verbatim}

---


  

Una volta effettuato lo shape dell'immagine caricata l'output è:

(208, 240, 256)


In [13]:
import xml.etree.ElementTree as ET
from pathlib import Path
import nibabel as nib
import torch
import pickle

# Funzione **extract**

Questa funzione prede due parametri:

    def extract(path_xml_patient, dict_Patient):

1. ***path_xml_patient*** si riferice al path del file *xml* che da cui si ricaveranno le informazioni.
2. ***dict_Patient*** dizionario contenente tutti i paziente e le loro relative RMI.

Nella porzione di codice seguente:

    if dict_Patient.get(subjectIdentifier, "None") == "None":
     dict_Patient[subjectIdentifier] = []
     dict_Patient[subjectIdentifier].append(imageUID)
    else:
      dict_Patient[subjectIdentifier].append(imageUID)

* Nel primo if si controlla se non esiste nel dict contenente i pazienti il **paziente** presente nel corretne file xml, nel caso in cui non eisista viene creata una lista nel dizionario con **chiave** proprio ***subjectIdentifier*** e viene caricato il codice del' MRI in questione.
* Nell'else nel caso in cui già esista il paziente viene aggiunto alla lsita l'UID del file xml corrente.

La **funzione** oltre a modificare il dizionario dei pazienti ritornerà età e codice dell'MRI, ***UID***.

In [14]:
def extract(path_xml_patient, dict_Patient):
  label = {}
  timepoint = {}

  tree = ET.parse(path_xml_patient)
  root = tree.getroot()

  subjectIdentifier = root.find('.//subjectIdentifier').text
  #print(f"Subject Identifier: {subjectIdentifier}")

  subjectAge = (root.find('.//subjectAge').text).split('.')[0]
  #print(f"Subject Age: {subjectAge}")

  imageUID = "I" + root.find('.//imageUID').text
  #print(f"Image UID: {imageUID}")

  label[imageUID] = subjectIdentifier
  timepoint[imageUID] = subjectAge

  if dict_Patient.get(subjectIdentifier, "None") == "None":
    dict_Patient[subjectIdentifier] = []
    dict_Patient[subjectIdentifier].append(imageUID)
  else:
    dict_Patient[subjectIdentifier].append(imageUID)

  return label, timepoint

# Creazione dei Dizionari

I dizioanri richeisti sono tre:
1. ***Data*** contenente i tensori dell'immagine.
2. ***Label*** contenente il codice univici dell'utente.
3. ***TimePoint*** contenente l'età del paziente al tempo del MRI.

Per ogni file xml presetne, si utilizza il primo ciclo for, si ottengono i valori dell'età e dei codici delle MRI, UID, i quali vengono ottenuti dalla funzione precedentemente descritta:

    label, age = extract(str(dir), dict_Patient)

Una volta ottenuti i nuovi valori vengono concatenati i vecchi dizionari e quelli tornati dalla funzione:

    dict_Label = {**dict_Label, **label}
    dict_TimePoint = {**dict_TimePoint, **age}

Lo scopo del secondo ciclo for è quello di ottenere i tensori per ogni immagine, per fare ciò sono necessari più passaggi:

1. Ottenere il file con l'estensione nii.gz, per cui sono necessari i tre cicli for annidati.
2. Utilizzando la libreria ***nibabel***, si ottiene l'immagine tridimensionale e i dati che le compongono.
3. I dati devono essere convertiti in torch tensor poiché sono in numby.ndarray.
4. Una volta fatto ciò si ottengono i tensori che compongono un'immagine, i quali verranno salvati all'interno del dizionario con la chiave ***uid*** che indica il codice della foto.

Di seguito il codice che effettua l'euristica appena menzioanta:

    nifti_path = str(image)
    nifti_image = nib.load(nifti_path)
    data = torch.from_numpy(nifti_image.get_fdata())
    dict_Data[uid] = data
  
Una volta ottenuti tutti i valori che ci servono per ogni immagine, si genera un unico dizionario, dove i dizionari prima di essere inseriti vengono ***ordinati**.

    dict_Save =  [dict(sorted(dict_Patient.items())),
                  dict(sorted(dict_Label.items())),
                  dict(sorted(dict_TimePoint.items())),
                  dict(sorted(dict_Data.items()))]

In fine tramite ***pickle*** il dizionario viene salvato per poi essere utilizzato successivamente.

In [15]:
metadata = Path("/content/drive/MyDrive/Colab_Notebooks/Tesi/Longitudinal_VAE/Dataset/MetaData")
dict_Label = {}
dict_Data = {}
dict_Patient = {}
dict_TimePoint = {}
for dir in metadata.iterdir():
  if dir.is_dir() == False and str(dir).split('/')[-1] != ".DS_Store":
    label, age = extract(str(dir), dict_Patient)
    dict_Label = {**dict_Label, **label}
    dict_TimePoint = {**dict_TimePoint, **age}

ninImage = Path('/content/drive/MyDrive/Colab_Notebooks/Tesi/Longitudinal_VAE/Dataset/Image_NiFTY_Longitudinal')
for dir in ninImage.iterdir():
  for patient in dir.iterdir():
    uid = str(patient).split('/')[-1]
    for image in patient.iterdir():
      nifti_path = str(image)
      nifti_image = nib.load(nifti_path)

      data = torch.from_numpy(nifti_image.get_fdata())
      dict_Data[uid] = data

dict_Save =  [dict(sorted(dict_Patient.items())),
              dict(sorted(dict_Label.items())),
              dict(sorted(dict_TimePoint.items())),
              dict(sorted(dict_Data.items()))]

with open("/content/drive/MyDrive/Colab_Notebooks/Tesi/Longitudinal_VAE/Dataset/dict.pkl", "wb") as file:
    pickle.dump(dict_Save, file)

# Per Caricare il dizionario

In [16]:
percorso_file = "/content/drive/MyDrive/Colab_Notebooks/Tesi/Longitudinal_VAE/Dataset/dict.pkl"

with open(percorso_file, 'rb') as file:
    dict_Load = pickle.load(file)

dict_Load

[{'002_S_1155': ['I995496', 'I1270004'],
  '002_S_1261': ['I989320', 'I1270020'],
  '002_S_4229': ['I1050345', 'I1270036'],
  '002_S_4654': ['I1174244', 'I1001975'],
  '002_S_4799': ['I1010814', 'I1270052']},
 {'I1001975': '002_S_4654',
  'I1010814': '002_S_4799',
  'I1050345': '002_S_4229',
  'I1174244': '002_S_4654',
  'I1270004': '002_S_1155',
  'I1270020': '002_S_1261',
  'I1270036': '002_S_4229',
  'I1270052': '002_S_4799',
  'I989320': '002_S_1261',
  'I995496': '002_S_1155'},
 {'I1001975': '81',
  'I1010814': '74',
  'I1050345': '73',
  'I1174244': '82',
  'I1270004': '70',
  'I1270020': '83',
  'I1270036': '74',
  'I1270052': '75',
  'I989320': '82',
  'I995496': '69'},
 {'I1001975': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,